In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Libraries
import pandas as pd
import os

from src.data import prep_data as prep
from src.models import model_schemata as schema
from src.models import build_model as build

In [3]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/ebdt_data/sub_network_e/p_fc.csv')

In [5]:
e_ksea_ad = prep.expand_ad_cpd(data['e_ksea'], 'value')
p_fc_ad = prep.expand_ad_cpd(data['p_fc'], 'value')
p_fc_ad

,phosphosite,sample,value,prob,p_dec,p_inc,p_base
0,ABL1(S569),AC220,dec,0.736520,0.736520,0.001000,0.262480
1,ABL1(S569),AC220,inc,0.736520,0.001000,0.736520,0.262480
2,ABL1(S569),AT13148,dec,0.215892,0.215892,0.001000,0.783108
3,ABL1(S569),AT13148,inc,0.215892,0.001000,0.215892,0.783108
4,ABL1(S569),AZ20,dec,0.057412,0.057412,0.001000,0.941588
...,...,...,...,...,...,...,...
1337,SRC(S75),U73122,inc,0.753525,0.001000,0.753525,0.245475
1338,SRC(S75),Ulixertinib,dec,0.564628,0.564628,0.001000,0.434372
1339,SRC(S75),Ulixertinib,inc,0.564628,0.001000,0.564628,0.434372
1340,SRC(S75),Vemurafenib,dec,0.515025,0.515025,0.001000,0.483975


In [6]:
# Mapping data to Problog predicates
# make sure to delete 'base' from schemata and AnnotatedDisjunctionClauseTemplate
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(e_ksea_ad, 'enzyme', 'sample', 'value')
predicates['e_ksea'].add_ad_probabilities(e_ksea_ad, p_dec_col='p_dec', p_base_col='p_base', p_inc_col='p_inc')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(p_fc_ad, 'phosphosite', 'sample', 'value')
predicates['p_fc'].add_ad_probabilities(p_fc_ad, p_dec_col='p_dec', p_base_col='p_base', p_inc_col='p_inc')

In [7]:
model_lfi = 'models/ebdt_data/sub_network/e_model/e_model_lfi_exp1.pl'
for predicate in predicates: 
    ad_generator = build.ProblogStatementGenerator(predicates[predicate])
    clauses = ad_generator.generate_ad_clauses(type='boolean')
    clauses.extend(ad_generator.generate_ad_clauses(type='domain'))
    build.insert_statements(model=model_lfi, statements=clauses, location='%% p1::{}'.format(predicate)) # insert into Problog file